In [ ]:
# clone the code repo,
!pip install git+git://github.com/depctg/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/depctg/nvcc4jupyter.git to /tmp/pip-req-build-3nsuwrm_
  Running command git clone -q git://github.com/depctg/nvcc4jupyter.git /tmp/pip-req-build-3nsuwrm_
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4334 sha256=b34eaf4fba1f50ae18ecabd80256472c3d134086fa38b148f9141c3546009ebb
  Stored in directory: /tmp/pip-ephem-wheel-cache-jogl49v_/wheels/1e/43/2d/099cad2b9b02dfa88573f50a22735d8a0b2ba69bf82167b81c
Successfully built NVCCPlugin
Default out bin result.out


In [ ]:
!lsb_release -a
!nvcc --version
!nvidia-smi

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sat Apr  3 15:11:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A

In [ ]:
%%writefile exercise01.cu
#include <stdio.h>
#include <assert.h>

#define MAX_ERR 0.1
#define MULTI_TIMES_RUN 1

__global__ void vector_dot_product(float *result, float *vector_a, float *vector_b, int vertor_length) 
{
    extern __shared__ float temp[];
 
    int index = threadIdx.x;    // index offset of this thread
    int stride = blockDim.x;    // stride step of each iteration

    // so if threadIdx.x=0, and blockDim.x=10,
    // then this thread is responsible for calculating temp[0], temp[10], temp[20]
    // similiarly, the following thread will calculate temp[1], temp[11], temp[21]
    for(int i = index; i < vertor_length; i += stride)
    {
        temp[i] = vector_a[i] * vector_b[i];
    }
 
    __syncthreads(); // synchronize all threads
 
    // The accumulation only needs to happen at thread_0
    if (threadIdx.x == 0)
    {
        float sum = 0;
        for (int i = 0; i < vertor_length; i++)
        {
            sum += temp[i];
        }
        *result=sum;
    }
}

int main(int argc, char *argv[])
{
    float *vector_a, *vector_b, *result;
    float *d_vector_a, *d_vector_b, *d_result;
    int list_of_thread_num[]={1,64,128,256,512,1024};
    int list_of_vector_length[]={100,200,1000,2000,10000};
    int thread_num = 1;
    int vector_length = 1000;
 
     if( argc == 3 ) {
      //printf("The argument supplied is %s\n", argv[1]);
      int arg1 = atoi(argv[1]);  //argv[0] is the program name
                                //atoi = ascii to int
      int arg2 = atoi(argv[2]);              
      
      vector_length = list_of_vector_length[arg1];
      thread_num = list_of_thread_num[arg2];
    }
    else if( argc > 2 ) {
      printf("Too many arguments supplied.\n");
    }
    else {
      printf("One argument expected.\n");
      
    }

    // Allocate memory on CPU
    vector_a = (float*)malloc(sizeof(float) * vector_length);
    vector_b = (float*)malloc(sizeof(float) * vector_length);
    result = (float*)malloc(sizeof(float));

    // data initializtion
    for(int i = 0; i < vector_length; i++)
    {
        vector_a[i] = 0.1f;
        vector_b[i] = 0.9f;
    }

    // Allocate memory on GPU
    cudaMalloc((void**)&d_vector_a, sizeof(float) * vector_length);
    cudaMalloc((void**)&d_vector_b, sizeof(float) * vector_length);
    cudaMalloc((void**)&d_result, sizeof(float));

    // copy operator to GPU
    cudaMemcpy(d_vector_a, vector_a, sizeof(float) * vector_length, cudaMemcpyHostToDevice);
    cudaMemcpy(d_vector_b, vector_b, sizeof(float) * vector_length, cudaMemcpyHostToDevice);

    // GPU do the work, CPU waits
#if MULTI_TIMES_RUN
    for(int i=0; i< 10; i++)
    {
#endif
        vector_dot_product<<<1,thread_num,sizeof(float) * vector_length>>>(d_result, d_vector_a, d_vector_b, vector_length);
#if MULTI_TIMES_RUN
    }
 #endif
 
    // Get results from the GPU
    cudaMemcpy(result, d_result, sizeof(float), 
               cudaMemcpyDeviceToHost);
 
    // Test the result
    //assert(fabs(*result - vector_length*2*3.14) < MAX_ERR);
    
    // you only need them for checking if the math is correct
     printf("result[0] = %f\n", result[0]);
    // printf("PASSED\n");

    // Free the memory
    cudaFree(d_vector_a);
    cudaFree(d_vector_b);
    cudaFree(d_result);
    free(vector_a);
    free(vector_a);
    free(result);

}

Overwriting exercise01.cu


In [ ]:
%%writefile exercise01.cu
#include <stdio.h>
#include <assert.h>

#define MAX_ERR 0.1
#define MULTI_TIMES_RUN 1

__global__ void vector_dot_product(float *result, float *vector_a, float *vector_b, int vertor_length) 
{
    extern __shared__ float temp[];
 
    int index = threadIdx.x;    // index offset of this thread
    int stride = blockDim.x;    // stride step of each iteration

    temp[threadIdx.x] = 0;
    for(int i = index; i < vertor_length; i += stride)
    {
        temp[threadIdx.x] = temp[threadIdx.x] + vector_a[i] * vector_b[i];
    }
 
    __syncthreads(); // synchronize all threads
 
    // The accumulation only needs to happen at thread_0
    if (threadIdx.x == 0)
    {
        float sum = 0;
        int thread_num = (vertor_length+blockDim.x)/blockDim.x;
        for (int i = 0; i < thread_num; i++)
        {
            sum += temp[i];
        }
        *result=sum;
    }
}

int main(int argc, char *argv[])
{
    float *vector_a, *vector_b, *result;
    float *d_vector_a, *d_vector_b, *d_result;
    int list_of_thread_num[]={1,64,128,256,512,1024};
    int list_of_vector_length[]={100,200,1000,2000,10000};
    int thread_num = 1;
    int vector_length = 1000;
 
     if( argc == 3 ) {
      //printf("The arguments supplied are %s, %s\n", argv[1], argv[2]);
      int arg1 = atoi(argv[1]);  //argv[0] is the program name
                                //atoi = ascii to int
      int arg2 = atoi(argv[2]);              
      
      vector_length = list_of_vector_length[arg1];
      thread_num = list_of_thread_num[arg2];
    }
    else if( argc > 2 ) {
      printf("Too many arguments supplied.\n");
    }
    else {
      printf("Two argument expected.\n");
      return 0;
    }

    // Allocate memory on CPU
    vector_a = (float*)malloc(sizeof(float) * vector_length);
    vector_b = (float*)malloc(sizeof(float) * vector_length);
    result = (float*)malloc(sizeof(float));

    // data initializtion
    for(int i = 0; i < vector_length; i++)
    {
        vector_a[i] = 0.1f;
        vector_b[i] = 0.9f;
    }

    // Allocate memory on GPU
    cudaMalloc((void**)&d_vector_a, sizeof(float) * vector_length);
    cudaMalloc((void**)&d_vector_b, sizeof(float) * vector_length);
    cudaMalloc((void**)&d_result, sizeof(float));

    // copy operator to GPU
    cudaMemcpy(d_vector_a, vector_a, sizeof(float) * vector_length, cudaMemcpyHostToDevice);
    cudaMemcpy(d_vector_b, vector_b, sizeof(float) * vector_length, cudaMemcpyHostToDevice);

    // GPU do the work, CPU waits
#if MULTI_TIMES_RUN
    for(int i=0; i< 10; i++)
    {
#endif
        vector_dot_product<<<1,thread_num,sizeof(float) * thread_num>>>(d_result, d_vector_a, d_vector_b, vector_length);
#if MULTI_TIMES_RUN
    }
 #endif
 
    // Get results from the GPU
    cudaMemcpy(result, d_result, sizeof(float), 
               cudaMemcpyDeviceToHost);
 
    // Test the result
    //assert(fabs(*result - vector_length*2*3.14) < MAX_ERR);
    
    // you only need them for checking if the math is correct
     printf("result[0] = %f\n", result[0]);
    // printf("PASSED\n");

    // Free the memory
    cudaFree(d_vector_a);
    cudaFree(d_vector_b);
    cudaFree(d_result);
    free(vector_a);
    free(vector_a);
    free(result);
}

Overwriting exercise01.cu


In [ ]:
!nvcc -o exercise01 exercise01.cu
!nvprof ./exercise01 0 0
!nvprof ./exercise01 1 0
!nvprof ./exercise01 2 0
!nvprof ./exercise01 3 0
!nvprof ./exercise01 4 0

==6575== NVPROF is profiling process 6575, command: ./exercise01 0 0
result[0] = 0.000000
==6575== Profiling application: ./exercise01 0 0
==6575== Warning: 13 records have invalid timestamps due to insufficient device buffer space. You can configure the buffer space using the option --device-buffer-size.
==6575== Warning: 3 records have invalid timestamps due to insufficient semaphore pool size. You can configure the pool size using the option --profiling-semaphore-pool-size.
==6575== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   83.05%  199.42ms         3  66.472ms  3.0070us  199.41ms  cudaMalloc
                   16.69%  40.071ms         3  13.357ms  12.754us  40.037ms  cudaMemcpy
                    0.14%  348.10us         1  348.10us  348.10us  348.10us  cuDeviceTotalMem
                    0.06%  155.68us       101  1.5410us     135ns  66.454us  cuDeviceGetAttribute
               

In [ ]:
!nvprof ./exercise01 

Two argument expected.
======== Warning: The path to CUPTI and CUDA Injection libraries might not be set in LD_LIBRARY_PATH. By default, CUPTI library is installed in /usr/local/<cuda-toolkit>/extras/CUPTI/lib64 or /usr/local/<cuda-toolkit>/targets/<arch>/lib and CUDA Injection library is installed in /usr/local/<cuda-toolkit>/targets/<arch>/lib
======== Warning: No profile data collected.


In [ ]:
!nvcc -o exercise01 exercise01.cu
!nvprof ./exercise01 4 0
!nvprof ./exercise01 4 1
!nvprof ./exercise01 4 2
!nvprof ./exercise01 4 3
!nvprof ./exercise01 4 4

==6453== NVPROF is profiling process 6453, command: ./exercise01 4 0
result[0] = 900.097656
==6453== Profiling application: ./exercise01 4 0
==6453== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.94%  20.735ms        10  2.0735ms  2.0733ms  2.0738ms  vector_dot_product(float*, float*, float*, int)
                    0.05%  9.6000us         2  4.8000us  4.6400us  4.9600us  [CUDA memcpy HtoD]
                    0.01%  2.2400us         1  2.2400us  2.2400us  2.2400us  [CUDA memcpy DtoH]
      API calls:   90.49%  207.43ms         3  69.142ms  3.6440us  207.42ms  cudaMalloc
                    9.06%  20.778ms         3  6.9259ms  25.878us  20.723ms  cudaMemcpy
                    0.23%  537.17us         1  537.17us  537.17us  537.17us  cuDeviceTotalMem
                    0.09%  208.51us       101  2.0640us     183ns  85.503us  cuDeviceGetAttribute
                    0.07%  153.48us         3  51.160us  5.1150us